# Project 3a - Planetary Motion

## Due 2/20

Particle motion under gravity and drag are useful test cases because they are quite simple.  However, there are only so many things that can occur because there is no interaction between different particles.  An interesting generalization of particle motion is the case in which we do not assume uniform gravity, but rather use the full specification for the gravitational force between two particles $i$ and $j$ (or more specifically, the force exerted on particle $i$ by particle $j$), which is given by
$$ F_{ij} = -G\frac{m_i m_j}{|\mathbf{r}_{ij}|^3} \mathbf{r}_{ij}, $$
where $\mathbf{r}_{ij} = \mathbf{x}_i - \mathbf{x}_j$, and $|\mathbf{r}_{ij}| = \sqrt{\mathbf{r}_{ij} \cdot \mathbf{r}_{ij}}$ is the distance between the two particles.  Note that this force is anti-symmetric (in the sense that the force on particle $i$ exerted by particle $j$ is the negative of the force exerted on $j$ by $i$ - the only difference is that $r_{12}=-r_{21}$), and leads to an acceleration in particle $i$ due to its interaction with particle $j$ (and vice versa) that look like this
$$
\frac{\partial \mathbf{v}_i}{\partial t} = -G \frac{m_j}{|\mathbf{r}_{ij}|^3} \mathbf{r}_{ij}.
$$
As before, when integrating Newton's laws of motion, the derivative of position is just velocity
$$
\frac{\partial \mathbf{x}_i}{\partial t} = \mathbf{v}_i.
$$

If we have $n$ particles, the forces are additive and we have that for a general particle $i$
$$
\frac{\partial \mathbf{x}_i}{\partial t} = \mathbf{v}_i.
$$
$$
\frac{\partial \mathbf{v}_i}{\partial t} = \sum_{k=1,k\neq i}^N -G \frac{m_k}{|\mathbf{r}_{ik}|^3} \mathbf{r}_{ik}.
$$

### Planetary motion problem
**Your first task is to create a planetary motion problem class using the same specification as before**.  Note that you will have to keep track of a few things: for each particle, you will have two coordinates and two velocities, for a total of $4N$ degrees of freedom.  You should be careful to choose an organizational structure that lets you keep these things straight. 

In [30]:
import numpy as np

class PlanetaryMotion:
    """ This is an example class for an ODE specification"""
    
    def __init__(self,G=6.67e-11,m=np.array([1.,1.]),n_bodies=2):
        self.G = G
        self.m = m
        self.n_bodies = n_bodies
        self.dof = 4*n_bodies # for each body we have 4 degrees of freedom (x,y,vx,vy)
        pass
        
    def rhs(self,t,u):
        # the right hand side of the ode (or $\mathcal{F}(t,u)$)
        # u is a 1D array of length dof, i.e. for 2 bodies looks like [x1,y1,x2,y2,vx1,vy1,vx2,vy2]
        dudt = np.zeros(self.dof)
        # velocity body 1
        dudt[0] = u[4] # dx1/dt = vx1
        dudt[1] = u[5] # dy1/dt = vy1
        # velocity body 2
        dudt[2] = u[6] # dx2/dt = vx2
        dudt[3] = u[7] # dy2/dt = vy2
        # acceleration body 1
        r = np.sqrt((u[0]-u[2])**2 + (u[1]-u[3])**2) # distance between the bodies
        dudt[4] = -self.G*self.m[0]*(u[0]-u[2])/r**3 # dvx1/dt = -G*m2*(x1-x2)/r^3
        dudt[5] = -self.G*self.m[0]*(u[1]-u[3])/r**3 # dvy1/dt = -G*m2*(y1-y2)/r^3
        # acceleration body 2
        dudt[6] = -self.G*self.m[1]*(u[2]-u[0])/r**3 # dvx2/dt = -G*m1*(x2-x1)/r^3
        dudt[7] = -self.G*self.m[1]*(u[3]-u[1])/r**3 # dvy2/dt = -G*m1*(y2-y1)/r^3
        return dudt
        pass

    def rhs_n_body(self,t,u):
        # the right hand side of the ode (or $\mathcal{F}(t,u)$)
        # u is a 1D array of length dof, i.e. for 2 bodies looks like [x1,y1,x2,y2,vx1,vy1,vx2,vy2]
        dudt = np.zeros(self.dof)
        # iterate over all bodies
        for i in range(self.n_bodies):
            # velocity body i
            dudt[2*i] = u[2*i+2]
            dudt[2*i+1] = u[2*i+3]
            # acceleration body i
            dudt[2*i+2] = 0
            dudt[2*i+3] = 0
            for j in range(self.n_bodies):
                if i != j:
                    r = np.sqrt((u[2*i]-u[2*j])**2 + (u[2*i+1]-u[2*j+1])**2) # distance between the bodies
                    dudt[2*i+2] += -self.G*self.m[j]*(u[2*i]-u[2*j])/r**3 # dvx/dt = -G*m2*(x1-x2)/r^3
                    dudt[2*i+3] += -self.G*self.m[j]*(u[2*i+1]-u[2*j+1])/r**3 # dvy/dt = -G*m2*(y1-y2)/r^3
        return dudt

[ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -1.6675e-11 -0.0000e+00
  1.6675e-11 -0.0000e+00]


### A conservative time stepping scheme
Unfortunately these equations are a bit picky: they are sensitive to accumulating errors in the system.  One way to account for this is to use a time-stepping scheme that conserves energy exactly: even if the integration isn't accurate, it will not blow up.  One such algorithm is the Euler-Cromer algorithm, which is given by Eq. 3.34 in Gould.  **Implement this time-stepping scheme**.  Note in particular that Euler-Cromer is a bit more annoying to implement than, say, a Runge-Kutta method because the velocities must be treated differently (i.e. updated first) than the positions (which are updated using the *already updated* velocities).  As such, I suggest creating two lists of indices in the __init__ method that keep track of which entries in the state vector are velocities and which are positions.  

In [9]:
class EulerCromer:
    def __init__(self):
        pass   
    
    def step(self,ode,t,dt,u_0):
        # take a single step using the Euler-Cromer method
        v = u_0[1] # velocity
        a = ode.rhs(t,u_0) # acceleration
        v_new = v + a*dt # update the velocity
        u_new = u_0 + v_new*dt # update the position
        return u_new

### Integrating the equations of motion
We can use the same Integrator object as before.  **Integrate the equations of motion for a 2 body system.**  Setting the initial conditions can be a little bit tricky: not all conditions lead to stable orbits.  One such circumstance that does is for $m_1=1e4$, $m_2=1$, and $G=1$, with the velocity and position of particle 1 $\mathbf{x}_1 = [0,0]$, and the velocity and position of particle 2 given by
$$
\mathbf{x}_2 = [100,0]
$$
$$
\mathbf{v}_2 = [0,10]
$$
**Plot the trajectory of the particle in whatever way seems suitable to you.** I find that making two plots is helpful, the first being position versus time for each particle and each dimension, as well as a plot of the two position variables.  

In [11]:
class Integrator:
    def __init__(self,ode,method):
        self.ode = ode
        self.method = method
        
    def integrate(self,interval,dt,u_0):
        t_0 = interval[0]
        t_end = interval[1]
        
        times = [t_0]
        states = [u_0]
        
        t = t_0
        while t<t_end:
            dt_ = min(dt,t_end-t)
            u_1 = self.method.step(self.ode,t,dt_,u_0)
            t = t + dt_
            u_0 = u_1
            
            times.append(t)
            states.append(u_1)
            
        return np.array(times),np.array(states)

### Energy Conservation
If you implemented the above correctly, your particle(s) should be moving in a stable, repeating, circular trajectory.  Now try again with Euler's method rather than Euler-Cromer.  What is happening here?  It turns out that Euler's method does not conserve energy.  We can illustrate this directly by computing the total energy of the system as a function of time.  There are two reservoirs of energy in this system: the gravitational potential energy, which is given by
$$
T = -\sum_{i=1}^N \sum_{k=1,i\neq k}^N G \frac{m_i m_k}{|\mathbf{r}_{ik}|}
$$
and the kinetic energy
$$
K = \sum_{i=1}^N \frac{1}{2} m_i v_i^2
$$
**Compute and plot $T + K$ at every time step for both the Euler-Cromer and Euler's method, and comment whether each method conserves energy**.


### 3-body problem
**Add a third particle to the system**.  It may require a little bit of trial and error to find a stable trajectory (HINT: One stable configuration would add a particle 3 that is somehow symmetric with particle 2.  Explore modifications to the mass and initial conditions to see what kinds of things can happen to our little planets.  

---